In [2]:
import os
import numpy as np
import math
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
data = input_data.read_data_sets('data/MNIST/', one_hot=True)
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(data.test.labels)))
print("- Validation-set:\t{}".format(len(data.validation.labels)))

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz
Size of:
- Training-set:		55000
- Test-set:		10000
- Validation-set:	5000


In [50]:
conv_layer1_filter_size = 5
conv_layer2_filter_size = 5
conv1_num_filters = 16
conv2_num_filters = 64
hidden_layer1_input_size = 7*7*64
hidden_layer1_size = 128
hidden_layer2_size = 64
image_size = 28
imageflat_size = image_size*image_size
num_channel = 1
num_classes = 10
batch_size = 100

In [92]:
def new_convolution_layer(input, filter_size, num_filters, num_channel, polling=True):
    shape = [filter_size, filter_size, num_channel, num_filters]
    weights = tf.Variable(tf.truncated_normal(shape=shape,stddev=0.05))
    #biases = tf.Variable(tf.truncated_normal(shape=[num_filters],stddev=0.05))
    layer = tf.nn.conv2d(input, filter = weights,strides=[1,1,1,1], padding = "SAME")
    #layer+= biases
    layer = tf.nn.max_pool(layer,[1,2,2,1],strides = [1,2,2,1], padding= "SAME")
    layer = tf.nn.relu(layer)
    return layer

def new_full_connected_layer(input, input_size, layer_size,bias =True):
    weights = tf.Variable(tf.truncated_normal(shape=(input_size, layer_size)))
    logit = tf.matmul(input,weights)
    if bias:
        biases = tf.Variable(tf.truncated_normal(shape=(1, layer_size)))
        logit += biases
    
    return logit



In [123]:
input = tf.placeholder(tf.float32, (None,imageflat_size))
labels = tf.placeholder(tf.float32, (None, num_classes))
is_training = tf.placeholder(tf.bool, (None))
input_image = tf.reshape(input,shape=(-1,image_size,image_size,num_channel))

conv_layer1 = new_convolution_layer(input_image, conv_layer1_filter_size, conv1_num_filters, num_channel)
conv_layer2 = new_convolution_layer(conv_layer1, conv_layer2_filter_size, conv2_num_filters, conv1_num_filters)
fc_input_shape = conv_layer2.shape[1]*conv_layer2.shape[2]*conv_layer2.shape[3]
flat_conv2_output = tf.reshape(conv_layer2,shape=[-1,fc_input_shape])

fc_hidden_layer1 = new_full_connected_layer(flat_conv2_output, hidden_layer1_input_size, hidden_layer1_size,bias = False)
fc_hidden_layer1 = tf.layers.dropout(fc_hidden_layer1, rate=0.5, training=is_training)
#fc_hidden_layer2 = new_full_connected_layer(tf.sigmoid(fc_hidden_layer1), hidden_layer1_size, hidden_layer2_size)
logits = new_full_connected_layer(tf.sigmoid(fc_hidden_layer1), hidden_layer1_size, num_classes)

nn_output = tf.nn.softmax(logits)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels)
cost=tf.reduce_sum(cross_entropy)

learning_rate = tf.placeholder(tf.float32,shape = [])
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

true_output = tf.argmax(labels,axis = 1)
output_nn = tf.argmax(nn_output, axis = 1)

correct_prediction = tf.equal(output_nn, true_output)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [124]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#sess.run(conv_layer1, feed_dict={input:[data.train.images[0]]}).shape

In [125]:
iter_count = 0
def fit(num_iterations,learningrate):
    global iter_count
    
    for i in range(iter_count,iter_count+num_iterations):
        input_next_batch, label_next_batch = data.train.next_batch(batch_size)
        feed_dict_train = {input:input_next_batch, labels:label_next_batch, learning_rate: learningrate,is_training : True}
        sess.run(optimizer,feed_dict=feed_dict_train)
        if i%100 == 0:
            feed_dict_train = {input:input_next_batch, labels:label_next_batch, learning_rate: learningrate,is_training: False}
            acc, loss= sess.run([accuracy, cost], feed_dict=feed_dict_train)
            print("Iteration {0}, Accuracy {1:.4f}%".format(i+1,acc*100))
    iter_count += num_iterations
feed_dict_test = {input:data.test.images[0:500], labels:data.test.labels[0:500],is_training: False}

In [126]:
print("Accuracy {}%".format(sess.run(accuracy, feed_dict= feed_dict_test)*100))
fit(500,0.001)
#print("Accuracy {}%".format(sess.run(accuracy, feed_dict= feed_dict_test)*100))

Accuracy 15.600000321865082%
Iteration 1, Accuracy 17.0000%
Iteration 101, Accuracy 89.0000%
Iteration 201, Accuracy 98.0000%
Iteration 301, Accuracy 95.0000%
Iteration 401, Accuracy 98.0000%


In [127]:
print("Accuracy {}%".format(sess.run(accuracy, feed_dict= feed_dict_test)*100))
fit(1000,0.001)

Accuracy 97.39999771118164%
Iteration 501, Accuracy 97.0000%
Iteration 601, Accuracy 100.0000%
Iteration 701, Accuracy 96.0000%
Iteration 801, Accuracy 98.0000%
Iteration 901, Accuracy 96.0000%
Iteration 1001, Accuracy 97.0000%
Iteration 1101, Accuracy 99.0000%
Iteration 1201, Accuracy 98.0000%
Iteration 1301, Accuracy 97.0000%
Iteration 1401, Accuracy 100.0000%


In [128]:
print("Accuracy {}%".format(sess.run(accuracy, feed_dict= feed_dict_test)*100))
fit(3000,0.001)

Accuracy 98.60000014305115%
Iteration 1501, Accuracy 100.0000%
Iteration 1601, Accuracy 100.0000%
Iteration 1701, Accuracy 100.0000%
Iteration 1801, Accuracy 99.0000%
Iteration 1901, Accuracy 100.0000%
Iteration 2001, Accuracy 99.0000%
Iteration 2101, Accuracy 100.0000%
Iteration 2201, Accuracy 100.0000%
Iteration 2301, Accuracy 99.0000%
Iteration 2401, Accuracy 100.0000%
Iteration 2501, Accuracy 99.0000%
Iteration 2601, Accuracy 98.0000%
Iteration 2701, Accuracy 100.0000%
Iteration 2801, Accuracy 100.0000%
Iteration 2901, Accuracy 98.0000%
Iteration 3001, Accuracy 99.0000%
Iteration 3101, Accuracy 100.0000%
Iteration 3201, Accuracy 99.0000%
Iteration 3301, Accuracy 99.0000%
Iteration 3401, Accuracy 100.0000%
Iteration 3501, Accuracy 98.0000%
Iteration 3601, Accuracy 100.0000%
Iteration 3701, Accuracy 100.0000%
Iteration 3801, Accuracy 98.0000%
Iteration 3901, Accuracy 99.0000%
Iteration 4001, Accuracy 99.0000%
Iteration 4101, Accuracy 100.0000%
Iteration 4201, Accuracy 100.0000%
Itera

In [129]:
print("Accuracy {}%".format(sess.run(accuracy, feed_dict= feed_dict_test)*100))

Accuracy 99.59999918937683%


In [ ]:
fit(3000,0.0001)